In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch
from PIL import Image
import torchvision.transforms as transforms
from torch.autograd import Variable
import torchvision

In [3]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm2d') != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [4]:
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()

        conv_block = [  nn.ReflectionPad2d(1),
                        nn.Conv2d(in_features, in_features, 3),
                        nn.InstanceNorm2d(in_features),
                        nn.ReLU(inplace=True),
                        nn.ReflectionPad2d(1),
                        nn.Conv2d(in_features, in_features, 3),
                        nn.InstanceNorm2d(in_features)  ]

        self.conv_block = nn.Sequential(*conv_block)

    def forward(self, x):
        return x + self.conv_block(x)

In [5]:
class GeneratorResNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, res_blocks=9):
        super(GeneratorResNet, self).__init__()

        # Initial convolution block
        model = [   nn.ReflectionPad2d(3),
                    nn.Conv2d(in_channels, 64, 7),
                    nn.InstanceNorm2d(64),
                    nn.ReLU(inplace=True) ]

        # Downsampling
        in_features = 64
        out_features = in_features*2
        for _ in range(2):
            model += [  nn.Conv2d(in_features, out_features, 3, stride=2, padding=1),
                        nn.InstanceNorm2d(out_features),
                        nn.ReLU(inplace=True) ]
            in_features = out_features
            out_features = in_features*2

        # Residual blocks
        for _ in range(res_blocks):
            model += [ResidualBlock(in_features)]

        # Upsampling
        out_features = in_features//2
        for _ in range(2):
            model += [  nn.ConvTranspose2d(in_features, out_features, 3, stride=2, padding=1, output_padding=1),
                        nn.InstanceNorm2d(out_features),
                        nn.ReLU(inplace=True) ]
            in_features = out_features
            out_features = in_features//2

        # Output layer
        model += [  nn.ReflectionPad2d(3),
                    nn.Conv2d(64, out_channels, 7),
                    nn.Tanh() ]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

In [25]:
class Discriminator(nn.Module):
    def __init__(self, in_channels=3):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, normalize=True):
            """Returns downsampling layers of each discriminator block"""
            layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)]
            if normalize:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers
        self.n1 = nn.Sequential(*discriminator_block(in_channels, 64, normalize=False))
        self.n2 = nn.Sequential(*discriminator_block(64, 128))
        self.n3 = nn.Sequential(*discriminator_block(128, 256))
        self.n4 = nn.Sequential(*discriminator_block(256, 512))
        self.n5 = nn.ZeroPad2d((1,0,1,0))
        self.n6 = nn.Conv2d(512, 1, 4, padding=1)

        self.model = nn.Sequential(
            *discriminator_block(in_channels, 64, normalize=False),
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(512, 1, 4, padding=1)
        )

    def forward(self, img):
        print(img.shape)
        x = self.n1(img)
        print(x.shape)
        x = self.n2(x)
        print(x.shape)
        x = self.n3(x)
        print(x.shape)
        x = self.n4(x)
        print(x.shape)
        x = self.n5(x)
        print(x.shape)
        x = self.n6(x)
        print(x.shape)
        #return self.model(img)
        return x

In [26]:
d = Discriminator()

In [30]:
img = Image.open("/home/arg_ws3/Pictures/Wallpapers/NickWang.jpg")
img_height = 480
img_width = 640
tfs = [ transforms.Resize((img_height, img_width), Image.BICUBIC),
                #transforms.RandomCrop((img_height, img_width)),
                #transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) ]
transforms_ = transforms.Compose(tfs)
item = transforms_(img)
item = Variable(item)

item = item.unsqueeze(0)
item.shape

torch.Size([1, 3, 480, 640])

In [31]:
d

Discriminator(
  (n1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace)
  )
  (n2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (2): LeakyReLU(negative_slope=0.2, inplace)
  )
  (n3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (2): LeakyReLU(negative_slope=0.2, inplace)
  )
  (n4): Sequential(
    (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (2): LeakyReLU(negative_slope=0.2, inplace)
  )
  (n5): ZeroPad2d(padding=(1, 0, 1, 0), value=0.0)
  (n6): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1),

In [32]:
out = d(item)
out.shape

torch.Size([1, 3, 480, 640])
torch.Size([1, 64, 240, 320])
torch.Size([1, 128, 120, 160])
torch.Size([1, 256, 60, 80])
torch.Size([1, 512, 30, 40])
torch.Size([1, 512, 31, 41])
torch.Size([1, 1, 30, 40])


torch.Size([1, 1, 30, 40])

In [9]:
dis = d(item)

In [10]:
dis.shape

torch.Size([1, 1, 15, 20])

In [11]:
class testNet(nn.Module):
    def __init__(self):
        super(testNet, self).__init__()
        self.base_model = torchvision.models.alexnet(pretrained = True)
        print(self.base_model)
        self.base_features = self.base_model.features
        for param in self.base_features.parameters():
            param.requires_grad = False
        # Everything except the last linear layer
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, (4, 4), (2, 2), (1,1)), #in, out, kernel, stride, padding
            nn.LeakyReLU(negative_slope=0.2, inplace=True)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1)), #in, out, kernel, stride, padding
            nn.InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False),
            nn.LeakyReLU(negative_slope=0.2, inplace=True)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1)), #in, out, kernel, stride, padding
            nn.InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False),
            nn.LeakyReLU(negative_slope=0.2, inplace=True)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1)), #in, out, kernel, stride, padding
            nn.InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.ZeroPad2d((1, 0, 1, 0))
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
        )
        self.new_classifier = nn.Sequential(
            nn.Linear(4099, 1)
        )

    def forward(self, x):
        x1 = self.conv1(x)
        print("conv1: ", x1.shape)
        x2 = self.conv2(x1)
        print("conv2: ", x2.shape)
        x3 = self.conv3(x2)
        print("conv3: ", x3.shape)
        x4 = self.conv4(x3)
        print("conv4: ", x4.shape)
        x5 = self.conv5(x4)
        print("conv5: ", x5.shape)
        f_out = self.base_features(x)
        #f_out = f_out.view(1, -1)
        print("features: ", f_out.shape)
        output = x5
        return output

In [12]:
test = testNet()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p=0.5)
    (4): Linear(in_features=4096, out_feature

In [13]:
t = test(item)

conv1:  torch.Size([1, 64, 120, 160])
conv2:  torch.Size([1, 128, 60, 80])
conv3:  torch.Size([1, 256, 30, 40])
conv4:  torch.Size([1, 512, 16, 21])
conv5:  torch.Size([1, 1, 15, 20])
features:  torch.Size([1, 256, 6, 9])


In [110]:
t.shape

torch.Size([1, 1, 15, 20])

tensor([[[[ 1.6396e-01,  1.3731e-01,  4.5993e-01,  2.6904e-01,  2.0481e-01,
            4.3343e-01, -2.4109e-02,  3.4406e-01, -9.8886e-02,  3.9919e-01,
            7.9689e-02,  2.9744e-01, -2.9480e-01, -4.1103e-01,  4.2609e-01,
            8.0310e-01,  7.1511e-01,  4.7087e-01,  2.0060e-01, -1.7292e-02],
          [ 2.2782e-01,  3.4933e-01,  5.1855e-01, -6.6352e-02,  2.6952e-02,
           -1.3521e-01, -6.2806e-01,  3.8714e-01,  1.9291e-02,  3.7754e-01,
            5.6412e-01, -3.0544e-02,  2.8047e-01,  4.7832e-01, -3.2351e-01,
            3.2049e-01,  5.5928e-01,  4.8534e-01, -2.1460e-02, -2.3494e-01],
          [ 2.2897e-01,  2.7126e-01,  2.1209e-01,  4.2137e-01, -2.4208e-01,
           -2.8636e-01, -5.5804e-02, -1.5814e-01, -3.5224e-01,  2.0652e-01,
            1.8558e-01,  3.2440e-01,  5.1743e-03,  1.5944e-01, -2.0050e-01,
           -9.7395e-02,  4.1364e-01,  3.8662e-01,  2.2204e-01, -2.6767e-01],
          [ 2.7021e-01,  9.5274e-02,  4.0826e-01, -2.5264e-01, -2.7920e-01,
         